In [1]:
import argparse
import os
import pathlib
import pprint
import sys

import pandas as pd

# Get from arg_parsing_utils import check_for_missing_args, parse_args
from notebook_init_utils import bandicoot_check, init_notebook

root_dir, in_notebook = init_notebook()
image_base_dir = bandicoot_check(
    pathlib.Path(os.path.expanduser("~/mnt/bandicoot")).resolve(), root_dir
)

In [2]:
patients_dir = pathlib.Path(f"{image_base_dir}/data/").resolve(
    strict=True
)  # directory containing patient folders

patient_list_file = pathlib.Path(f"{root_dir}/data/patient_IDs.txt").resolve(
    strict=True
)
patients = pd.read_csv(patient_list_file, header=None)[0].tolist()

# get a list of patient directories
patient_dirs = [d for d in patients_dir.iterdir() if d.is_dir() and d.name in patients]

patient_dirs.sort()

In [3]:
# get a list of the well_fov directories for each patient
well_fov_dirs = [
    x
    for patient_dir in patient_dirs
    for x in pathlib.Path(
        f"{patient_dir}/2D_analysis/1b.middle_slice_illum_correction"
    ).iterdir()
    if x.is_dir()
]
well_fov_dirs.sort()
well_fov_df = pd.DataFrame(well_fov_dirs, columns=["dir_path"])
well_fov_df["patient"] = well_fov_df["dir_path"].apply(
    lambda x: str(x.parent).split("/")[-3]
)
well_fov_df["well_fov"] = well_fov_df["dir_path"].apply(lambda x: x.stem)
well_fov_df

,dir_path,patient,well_fov
0,/home/lippincm/mnt/bandicoot/NF1_organoid_data...,NF0014_T1,C10-1
1,/home/lippincm/mnt/bandicoot/NF1_organoid_data...,NF0014_T1,C10-2
2,/home/lippincm/mnt/bandicoot/NF1_organoid_data...,NF0014_T1,C11-1
3,/home/lippincm/mnt/bandicoot/NF1_organoid_data...,NF0014_T1,C11-2
4,/home/lippincm/mnt/bandicoot/NF1_organoid_data...,NF0014_T1,C2-1
...,...,...,...
3716,/home/lippincm/mnt/bandicoot/NF1_organoid_data...,SARCO361_T1,G9-3
3717,/home/lippincm/mnt/bandicoot/NF1_organoid_data...,SARCO361_T1,G9-4
3718,/home/lippincm/mnt/bandicoot/NF1_organoid_data...,SARCO361_T1,G9-5
3719,/home/lippincm/mnt/bandicoot/NF1_organoid_data...,SARCO361_T1,G9-6


In [15]:
present_files = 0
missing_files = 0
missing_files_list = []
for index, row in well_fov_df.iterrows():
    patient = row["patient"]
    well_fov = row["well_fov"]
    max_z_slice_dir_to_check = pathlib.Path(
        f"{root_dir}/data/{patient}/2D_analysis/2a.cellprofiler_zmax_proj_output/{well_fov}/"
    ).resolve()
    middle_slice_dir_to_check = pathlib.Path(
        f"{root_dir}/data/{patient}/2D_analysis/2b.cellprofiler_middle_slice_output/{well_fov}/"
    ).resolve()
    middle_n_slice_dir_to_check = pathlib.Path(
        f"{root_dir}/data/{patient}/2D_analysis/2c.cellprofiler_middle_n_slice_max_proj_output/{well_fov}/"
    ).resolve()
    if not max_z_slice_dir_to_check.is_dir():
        missing_files += 1
        missing_files_list.append(str(max_z_slice_dir_to_check))
    else:
        present_files += 1
    if not middle_slice_dir_to_check.is_dir():
        missing_files += 1
        missing_files_list.append(str(middle_slice_dir_to_check))
    else:
        present_files += 1
    if not middle_n_slice_dir_to_check.is_dir():
        missing_files += 1
        missing_files_list.append(str(middle_n_slice_dir_to_check))
    else:
        present_files += 1

In [16]:
reruns_df = pd.DataFrame(missing_files_list, columns=["dir_path"])
reruns_df["patient"] = reruns_df["dir_path"].apply(lambda x: str(x).split("/")[-3])
reruns_df["well_fov"] = reruns_df["dir_path"].apply(lambda x: str(x).split("/")[-1])
reruns_df.drop(columns=["dir_path"], inplace=True)
reruns_df.drop_duplicates(inplace=True)
pathlib.Path("../loadfiles/").mkdir(parents=True, exist_ok=True)
reruns_df.to_csv(
    f"../loadfiles/featurization_loadfile.txt", index=False, sep="\t", header=False
)

In [18]:
print(f"Total directories checked: {len(well_fov_df) * 3}")
print(f"Present directories: {present_files}")
print(f"Missing directories: {missing_files}")
print("Missing directories list:")
if missing_files < 50:
    pprint.pprint(missing_files_list)

Total directories checked: 11163
Present directories: 11160
Missing directories: 3
Missing directories list:
['/home/lippincm/4TB_A/NF1_2D_organoid_profiling_pipeline/data/NF0016_T1/2D_analysis/2a.cellprofiler_zmax_proj_output/D11-2',
 '/home/lippincm/4TB_A/NF1_2D_organoid_profiling_pipeline/data/NF0016_T1/2D_analysis/2b.cellprofiler_middle_slice_output/D11-2',
 '/home/lippincm/4TB_A/NF1_2D_organoid_profiling_pipeline/data/NF0016_T1/2D_analysis/2c.cellprofiler_middle_n_slice_max_proj_output/D11-2']
